In [1]:
!pip install -q python-telegram-bot sentence-transformers groq nest-asyncio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.0/731.0 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 9.4 MB/s eta 0:00:00


In [2]:
# Import libraries
import numpy as np
import nest_asyncio
import asyncio
from sentence_transformers import SentenceTransformer
from groq import Groq
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, ContextTypes

nest_asyncio.apply()

In [3]:
# Replace with your actual tokens
TELEGRAM_TOKEN = "TELEGRAM TOKEN PASTE IT"
GROQ_API_KEY = "GROQ API PASTE IT"  # Get free key from https://console.groq.com


In [11]:
docs = {
    "policy": """Company Leave Policy:
Employees are entitled to 20 paid leaves per year.
Sick leave can be taken anytime with medical proof.
Remote work allowed up to 3 days per week.""",

    "faq": """FAQ:
Q: What is the refund policy?
A: Refunds are allowed within 30 days of purchase.

Q: Do you provide tech support?
A: Yes, 24/7 email support is available.""",

    "recipes": """Recipe: Pancakes
Ingredients: Flour, Milk, Eggs, Sugar, Butter
Steps: Mix ingredients, heat pan, cook both sides."""
}


In [12]:
print("Loading embedding model...")
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

print("Initializing Groq client...")
groq_client = Groq(api_key=GROQ_API_KEY)

Loading embedding model...
Initializing Groq client...


In [13]:
# ============================================
# Build Vector Store
# ============================================

def build_store(docs):
    texts, vectors = [], []
    for t in docs.values():
        # Split into chunks of 300 characters
        for i in range(0, len(t), 300):
            chunk = t[i:i+300]
            texts.append(chunk)
            vectors.append(embedding_model.encode(chunk))

    return texts, np.vstack(vectors)

print("Building vector store...")
store_texts, store_vectors = build_store(docs)
print(f"Vector store ready with {len(store_texts)} chunks")


Building vector store...
Vector store ready with 3 chunks


In [14]:
# ============================================
# RAG Functions
# ============================================

def retrieve(query, k=3):
    """Retrieve top-k most relevant document chunks"""
    q = embedding_model.encode(query)
    sims = store_vectors @ q / (np.linalg.norm(store_vectors, axis=1) * np.linalg.norm(q))
    idx = np.argsort(sims)[-k:][::-1]
    return [store_texts[i] for i in idx]

In [15]:
def generate_answer(context, question):
    """Generate answer using Groq LLM"""
    try:
        prompt = f"""You are a helpful assistant. Answer the question based on the provided context.

Context:
{context}

Question: {question}

Answer the question clearly and concisely based only on the context provided."""

        response = groq_client.chat.completions.create(
            model="llama-3.3-70b-versatile",  # Fast and accurate model
            messages=[
                {"role": "system", "content": "You are a helpful assistant that answers questions based on provided context."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=500
        )

        return response.choices[0].message.content.strip()

    except Exception as e:
        return f"Error generating answer: {str(e)}"

In [16]:
# ============================================
# Telegram Bot Handlers
# ============================================

history = {}

async def start_cmd(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Handle /start command"""
    welcome_msg = """Welcome to the RAG Bot!

I can answer questions about:
• Company leave policy
• FAQ (refunds, tech support)
• Recipes

Commands:
/ask <your question> - Ask me anything
/help - Show this help message

Example: /ask what is the refund policy?"""

    await update.message.reply_text(welcome_msg)

async def help_cmd(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Handle /help command"""
    help_msg = """How to use this bot:

/ask <question> - Ask the bot a question
/help - Show this help message

Example questions:
• /ask what is the refund policy?
• /ask how many leaves do employees get?
• /ask how to make pancakes?"""

    await update.message.reply_text(help_msg)

async def ask_cmd(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Handle /ask command"""
    try:
        user = update.effective_user.id
        query = " ".join(context.args)

        # Check if query is empty
        if not query:
            await update.message.reply_text(
                "Please provide a question!\n\nExample: /ask what is the refund policy?"
            )
            return

        print(f"User {user} asked: {query}")

        # Send thinking message
        thinking_msg = await update.message.reply_text("Searching and generating answer...")

        # Retrieve relevant documents
        docs = retrieve(query)
        ctx = "\n\n".join(docs)

        print(f"Retrieved {len(docs)} relevant chunks")

        # Generate answer using Groq
        answer = generate_answer(ctx, query)

        # Store in history
        history.setdefault(user, []).append((query, answer))
        history[user] = history[user][-5:]  # Keep last 5 interactions

        # Send answer
        await thinking_msg.edit_text(f"💬 **Answer:**\n\n{answer}")

        print(f"Answered user {user}")

    except Exception as e:
        error_msg = f"Sorry, an error occurred: {str(e)}"
        print(f"Error: {e}")
        await update.message.reply_text(error_msg)

In [ ]:
# ============================================
# Bot Setup and Run
# ============================================

# Build application
app = ApplicationBuilder().token(TELEGRAM_TOKEN).build()

# Add command handlers
app.add_handler(CommandHandler("start", start_cmd))
app.add_handler(CommandHandler("help", help_cmd))
app.add_handler(CommandHandler("ask", ask_cmd))

async def run_bot():
    """Run the bot with manual polling"""
    print("Bot starting...")
    await app.initialize()
    await app.start()
    print("Bot is now running!")
    print("Send /start to your bot on Telegram to begin")
    print("\nPress Ctrl+C in Colab to stop the bot\n")

    offset = None
    try:
        while True:
            updates = await app.bot.get_updates(offset=offset, timeout=10)
            for u in updates:
                offset = u.update_id + 1
                await app.process_update(u)
            await asyncio.sleep(0.5)
    except KeyboardInterrupt:
        print("\nStopping bot...")
        await app.stop()
        print("Bot stopped")

# Run the bot
await run_bot()